# Deep Learning

## Assignment 3

Previously in `2_fullyconnected.ipynb`, we trained a logistic regression and a neural network model.

In this assignment, we're going to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`

In [3]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Valid set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Valid set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))) / predictions.shape[0]

### Problem 1

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, L2 loss for a tensor `t` can be computed using `nn.l2_loss(t)`. The right amount of the regularization should improve our validation / test accuracy.

Let's try with Linear Regression

In [31]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
        # L2 regularization for weights and normalized
        0.01 * tf.nn.l2_loss(weights)
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [32]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been normalized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 2000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 43.989449
Minibatch accuracy: 16.4%
Validation accuracy: 14.0%
Minibatch loss at step 2000: 6.565395
Minibatch accuracy: 75.8%
Validation accuracy: 76.0%
Minibatch loss at step 4000: 1.729037
Minibatch accuracy: 82.0%
Validation accuracy: 80.7%
Minibatch loss at step 6000: 0.966317
Minibatch accuracy: 82.8%
Validation accuracy: 82.0%
Minibatch loss at step 8000: 0.614404
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
Minibatch loss at step 10000: 0.695290
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 12000: 0.599478
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 14000: 0.901585
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 16000: 0.670599
Minibatch accuracy: 84.4%
Validation accuracy: 82.5%
Minibatch loss at step 18000: 0.821110
Minibatch accuracy: 78.9%
Validation accuracy: 82.4%
Minibatch loss at step 20000: 0.626464
Minibatch accuracy: 84.4%
Validation

For my case, a learning rate $\alpha = 0.04$ and a L2 regularization normalized coefficient of $0.01$ would give the best test accuracy of $89.1\%$ 

** Let's give another try with a simple 1-hidden layer fully connected neural network **

In [46]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    hidden_weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    out_weights = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    out_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    logits = tf.matmul(layer1, out_weights) + out_biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
        # L2 regularization
        0.001 * tf.nn.l2_loss(hidden_weights) + 
        0.001 * tf.nn.l2_loss(out_weights))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    layer1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(
        tf.matmul(layer1_valid, out_weights) + out_biases)
    
    layer1_test = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(layer1_test, out_weights) + out_biases)

In [47]:
num_steps = 40001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick an offset within the training data, which has been normalized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 4000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%'  % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 660.742310
Minibatch accuracy: 7.8%
Validation accuracy: 18.6%
Minibatch loss at step 4000: 229.919373
Minibatch accuracy: 82.0%
Validation accuracy: 82.1%
Minibatch loss at step 8000: 164.704041
Minibatch accuracy: 88.3%
Validation accuracy: 82.5%
Minibatch loss at step 12000: 118.644714
Minibatch accuracy: 90.6%
Validation accuracy: 84.3%
Minibatch loss at step 16000: 86.056305
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 20000: 62.431862
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 24000: 45.428196
Minibatch accuracy: 89.8%
Validation accuracy: 86.3%
Minibatch loss at step 28000: 32.934616
Minibatch accuracy: 92.2%
Validation accuracy: 86.8%
Minibatch loss at step 32000: 24.060024
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Minibatch loss at step 36000: 17.600689
Minibatch accuracy: 90.6%
Validation accuracy: 88.1%
Minibatch loss at step 40000: 13.003156
Minibatch accuracy: 8

In the above experiment, the best value for $\alpha =$